In [0]:
"""
Reverse product_id order in reverse like below.

Input
+----------+------------+-----------+
|product_id|product_name|   category|
+----------+------------+-----------+
|         1|      Laptop|Electronics|. 
|         2|  Smartphone|Electronics|
|         3|      Tablet|Electronics|
|         9|     Printer|Electronics|
|         4|  Headphones|Accessories|
|         5|  Smartwatch|Accessories|
|         6|    Keyboard|Accessories|
|         7|       Mouse|Accessories|
|         8|     Monitor|Accessories|
+----------+------------+-----------+

Output
+----------+------------+-----------+
|product_id|product_name|   category|
+----------+------------+-----------+
|         9|      Laptop|Electronics|. 
|         3|  Smartphone|Electronics|
|         2|      Tablet|Electronics|
|         1|     Printer|Electronics|
|         8|  Headphones|Accessories|
|         7|  Smartwatch|Accessories|
|         6|    Keyboard|Accessories|
|         5|       Mouse|Accessories|
|         4|     Monitor|Accessories|
+----------+------------+-----------+
"""

product_details_df = spark.createDataFrame(
    [
        (1, 'Laptop', 'Electronics'),
        (2, 'Smartphone', 'Electronics'),
        (3, 'Tablet', 'Electronics'),
        (4, 'Headphones', 'Accessories'),
        (5, 'Smartwatch', 'Accessories'),
        (6, 'Keyboard', 'Accessories'),
        (7, 'Mouse', 'Accessories'),
        (8, 'Monitor', 'Accessories'),
        (9, 'Printer', 'Electronics')
    ],
    ["product_id", "product_name", "category"]
)

product_details_df.show()

+----------+------------+-----------+
|product_id|product_name|   category|
+----------+------------+-----------+
|         1|      Laptop|Electronics|
|         2|  Smartphone|Electronics|
|         3|      Tablet|Electronics|
|         4|  Headphones|Accessories|
|         5|  Smartwatch|Accessories|
|         6|    Keyboard|Accessories|
|         7|       Mouse|Accessories|
|         8|     Monitor|Accessories|
|         9|     Printer|Electronics|
+----------+------------+-----------+



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

product_details_rearranged_df = product_details_df \
    .withColumn("asc_row", row_number().over(Window.partitionBy("category").orderBy("product_id"))) \
    .withColumn("desc_row", row_number().over(Window.partitionBy("category").orderBy(desc("product_id")))) \
    .orderBy("category", "product_id")

product_details_rearranged_df.show()

+----------+------------+-----------+-------+--------+
|product_id|product_name|   category|asc_row|desc_row|
+----------+------------+-----------+-------+--------+
|         4|  Headphones|Accessories|      1|       5|
|         5|  Smartwatch|Accessories|      2|       4|
|         6|    Keyboard|Accessories|      3|       3|
|         7|       Mouse|Accessories|      4|       2|
|         8|     Monitor|Accessories|      5|       1|
|         1|      Laptop|Electronics|      1|       4|
|         2|  Smartphone|Electronics|      2|       3|
|         3|      Tablet|Electronics|      3|       2|
|         9|     Printer|Electronics|      4|       1|
+----------+------------+-----------+-------+--------+



In [0]:
product_details_rearranged_df.alias("t1").join(product_details_rearranged_df.alias("t2"), (col("t1.category") == col("t2.category")) & (col("t1.asc_row") == col("t2.desc_row")), "inner") \
    .select("t2.product_id", "t1.product_name", "t1.category") \
    .show()

+----------+------------+-----------+
|product_id|product_name|   category|
+----------+------------+-----------+
|         8|  Headphones|Accessories|
|         7|  Smartwatch|Accessories|
|         6|    Keyboard|Accessories|
|         5|       Mouse|Accessories|
|         4|     Monitor|Accessories|
|         9|      Laptop|Electronics|
|         3|  Smartphone|Electronics|
|         2|      Tablet|Electronics|
|         1|     Printer|Electronics|
+----------+------------+-----------+



### SPARK SQL

In [0]:
product_details_df.createOrReplaceTempView("product_details")

spark.sql("""
          with cte as (
            select 
                *,
                row_number() over(partition by category order by product_id asc) as asc_row,
                row_number() over(partition by category order by product_id desc) as desc_row
            from product_details
            order by category, product_id    
          )
          select
            c2.product_id, c1.product_name, c1.category
          from cte c1 inner join cte c2 
          on c1.category = c2.category and c1.asc_row = c2.desc_row
          """).show()

+----------+------------+-----------+
|product_id|product_name|   category|
+----------+------------+-----------+
|         8|  Headphones|Accessories|
|         7|  Smartwatch|Accessories|
|         6|    Keyboard|Accessories|
|         5|       Mouse|Accessories|
|         4|     Monitor|Accessories|
|         9|      Laptop|Electronics|
|         3|  Smartphone|Electronics|
|         2|      Tablet|Electronics|
|         1|     Printer|Electronics|
+----------+------------+-----------+

